In [ ]:
import datetime
import numpy as np
from random import randint
!pip install cplex
import cplex
import math

     |████████████████████████████████| 31.0MB 103kB/s 


# **Part 1: Simplex Method for Linear Program (LP)**

#### **1)** Generate input matrices of standard linear program in matrix form.

##### Function

In [ ]:
def generate_matrix(m=False, n=False):
  # auto generate m, n
  if not m or not n:
    m, n = randint(2, 100), randint(2, 100)

  # random matrix size m*n with range [-50, 50]
  N = 100 * np.random.rand(m, n) - 50

  # create m*m matrix with ones on the diagonal and zeros elsewhere.
  B = np.eye(m)

  # create indices array
  N_ind = np.arange(1, n+1)
  B_ind = np.arange(n + 1, n+m+1)

  # create basic variables and non-basic variables
  b = 100 * np.random.rand(m, 1) - 50
  c_N = 100 * np.random.rand(n, 1) - 50

  return B, N, B_ind, N_ind, b, c_N

##### Example

In [ ]:
B, N, B_ind, N_ind, b, c_N = generate_matrix(2, 3)
print('B:')
print(B)
print('N:')
print(N)
print('b:')
print(b)
print('c_N:')
print(c_N)
print('B_ind:', B_ind)
print('N_ind:', N_ind)

B:
[[1. 0.]
 [0. 1.]]
N:
[[ 18.30661876 -41.59634101 -16.68227209]
 [-26.50079706   4.89171162  38.8543091 ]]
b:
[[-5.70003596]
 [-9.2037884 ]]
c_N:
[[35.13702477]
 [30.0752019 ]
 [41.15109843]]
B_ind: [4 5]
N_ind: [1 2 3]


#### **2)** Write a code to solve the generated LP using two phase simplex method in matrix form.

##### Function

In [ ]:
def two_phase_simplex_method(B, N, B_ind, N_ind, b, c_N):
  B = B.copy()
  N = N.copy()
  B_ind = B_ind.copy()
  N_ind = N_ind.copy()
  b = b.copy()
  c_N = c_N.copy()

  # get size matrix
  m, n = N.shape
  # x_B = b
  x_B = b
  # z_N = -c_N
  z_N = -c_N

  if (x_B < 0).sum() == 0:    # check if x_B have negasitive number
    try:
      B, N, B_ind, N_ind, x_B, z_N = primal_simplex(B, N, B_ind, N_ind, x_B, z_N)
    except:
      return {'status':'Unbounded', 'x':None}
  elif (z_N < 0).sum() == 0:  # check if z_N have negasitive number
    try:
      B, N, B_ind, N_ind, x_B, z_N = dual_simplex(B, N, B_ind, N_ind, x_B, z_N)
    except:
      return {'status':'Unbounded', 'x':None}
  else:                       # two phase
    z_N = np.ones((n, 1))
    try:
      B, N, B_ind, N_ind, x_B, z_N = dual_simplex(B, N, B_ind, N_ind, x_B, z_N)
    except:
      return {'status':'Infeasible', 'x':None}

    c_N_new = np.array([c_N[i-1][0] if i <=n else 0 for i in N_ind]).reshape(-1, 1)
    c_B_new = np.array([c_N[i-1][0] if i <=n else 0 for i in B_ind]).reshape(-1, 1)
    z_N = np.linalg.inv(B).dot(N).T.dot(c_B_new) - c_N_new
    try:
      B, N, B_ind, N_ind, x_B, z_N = primal_simplex(B, N, B_ind, N_ind, x_B, z_N)
    except:
      return {'status':'Unbounded', 'x':None}

  sol = {i: j for i, j in zip(B_ind.flatten(), x_B.flatten())}
  x = np.array([sol.get(i, 0) for i in np.arange(1, n+1)])
  return {'status':'Optimal', 'x':x}


def primal_simplex(B, N, B_ind, N_ind, x_B, z_N):
  m, n = N.shape
  while(True):
    try:
      j = np.where(z_N < 0)[0][0]
    except:
      break
    
    # calculate denta_x_B
    e_j = np.zeros((n, 1))
    e_j[j][0] = 1
    denta_x_B = np.linalg.inv(B).dot(N).dot(e_j)

    # find i, t
    i = np.argmax(denta_x_B / (x_B + 10**-6))
    t = x_B[i][0] / denta_x_B[i][0]
    if t < 0:
      # unbounded
      return None

    # find s, denta_z_N
    e_i = np.zeros((m, 1))
    e_i[i][0] = 1
    denta_z_N = - (np.linalg.inv(B).dot(N)).T.dot(e_i)
    s = z_N[j][0] / denta_z_N[j][0]

    # update x_B, z_N
    x_B = x_B - t * denta_x_B
    x_B[i][0] = t
    z_N = z_N - s * denta_z_N
    z_N[j][0] = s

    # create new sets of basic and nonbasic indices
    temp = B_ind[i]
    B_ind[i] = N_ind[j]
    N_ind[j] = temp
    temp = np.array(B[:, i])
    B[:, i] = np.array(N[:, j])
    N[:, j] = temp

  return B, N, B_ind, N_ind, x_B, z_N


def dual_simplex(B, N, B_ind, N_ind, x_B, z_N):
  m, n = N.shape
  while(True):
    try:
      i = np.where(x_B < 0)[0][0]
    except:
      break
    
    # calculate denta_z_N
    e_i = np.zeros((m, 1))
    e_i[i][0] = 1
    denta_z_N = - (np.linalg.inv(B).dot(N)).T.dot(e_i)

    # find s, j
    j = np.argmax(denta_z_N / (z_N + 10**-6))
    s = z_N[j][0] / denta_z_N[j][0]
    if s < 0:
      # unbounded
      return None

    # find t, denta_x_B
    e_j = np.zeros((n, 1))
    e_j[j][0] = 1
    denta_x_B = np.linalg.inv(B).dot(N).dot(e_j)
    t = x_B[i][0] / denta_x_B[i][0]

    # update x_B, z_N
    x_B = x_B - t * denta_x_B
    x_B[i][0] = t
    z_N = z_N - s * denta_z_N
    z_N[j][0] = s

    # create new sets of basic and nonbasic indices
    temp = B_ind[i]
    B_ind[i] = N_ind[j]
    N_ind[j] = temp
    temp = np.array(B[:, i])
    B[:, i] = np.array(N[:, j])
    N[:, j] = temp

  return B, N, B_ind, N_ind, x_B, z_N

##### Example
![alt text](https://github.com/tadangkhoa1999/Simplex-Method-Python-example/blob/master/img/example_1.png?raw=true)

In [ ]:
m, n = 2, 3
B = np.array([[1, 0], [0, 1]])
N = np.array([[-1, -1, -1], [2, -1, 1]])
N_ind = np.arange(1, n+1)
B_ind = np.arange(n + 1, n+m+1)
x_B = np.array([[-2], [1]])
z_N = np.array([[-2], [6], [0]])

# test two_phase_simplex_method
sol = two_phase_simplex_method(B, N, B_ind, N_ind, x_B, -z_N)
if sol['status'] == 'Optimal':
  print(sol['x'])
else:
  print(sol['status'])

[0.  0.5 1.5]


#### **3)** Solve the generated LP by a tool such as CVX, Cplex, ... to check your code.

##### Function

In [ ]:
def cplex_simplex_method(N, x_B, c_N):
  # The Linear Programming problem displayed bellow is as:
  #                  min z = c.x
  #    subject to:      Ax = b
  num_constraints, num_decision_var = N.shape
  A = N.astype(np.float).tolist()
  b = x_B.flatten().astype(np.float).tolist()
  c = (-c_N.flatten().astype(np.float)).tolist()

  # Establish the Linear Programming Model
  myProblem = cplex.Cplex()
  myProblem = cplex.Cplex()
  myProblem.set_log_stream(None)
  myProblem.set_error_stream(None)
  myProblem.set_warning_stream(None)
  myProblem.set_results_stream(None)
  # Add the decision variables and set their lower bound and upper bound (if necessary)
  myProblem.variables.add(names= ["x"+str(i) for i in range(num_decision_var)])
  for i in range(num_decision_var):
    myProblem.variables.set_lower_bounds(i, 0.0)

  # Add constraints
  for i in range(num_constraints):
    myProblem.linear_constraints.add(
      lin_expr= [cplex.SparsePair(ind= [j for j in range(num_decision_var)], val= A[i])],
      rhs= [b[i]],
      names = ["c"+str(i)],
      senses = ["L"]
    )

  # Add objective function and set its sense
  for i in range(num_decision_var):
    myProblem.objective.set_linear([(i, c[i])])
  myProblem.objective.set_sense(myProblem.objective.sense.minimize)
  # Solve the model and print the answer
  myProblem.solve()
  optimal_sol = myProblem.solution.get_values()
  return optimal_sol

##### Example

In [ ]:
sol = cplex_simplex_method(N, x_B, -z_N)
print('='*20)
print(sol)

[0.0, 0.5, 1.5]


#### **4)** Repeat (1)-(3) one hundred times and compare the mean and standard deviation of rune time of your code with those of the chosen tool.

##### Function

In [ ]:
mycode_time = []
tool_time = []
for i in range(100):
  B, N, B_ind, N_ind, b, c_N = generate_matrix()

  begin_time = datetime.datetime.now().timestamp()
  sol = two_phase_simplex_method(B, N, B_ind, N_ind, b, c_N)
  mycode_time.append(datetime.datetime.now().timestamp() - begin_time)
  begin_time = datetime.datetime.now().timestamp()
  try:
    optimal_sol,c_optimal_val = cplex_simplex_method(N, b, c_N)
  except:
    c_optimal_val = None
  tool_time.append(datetime.datetime.now().timestamp() - begin_time)

##### Result

In [ ]:
print("My code mean time:", np.mean(mycode_time))
print("My code standard deviation:", np.std(mycode_time))
print("Cplex tool mean time:", np.mean(tool_time))
print("Cplex tool standard deviation:", np.std(tool_time))

My code mean time: 0.17922647476196288
My code standard deviation: 0.2850430592997775
Cplex tool mean time: 0.01003655195236206
Cplex tool standard deviation: 0.00760154565338687


# **Part 2: Game theory**

#### ![alt text](https://github.com/tadangkhoa1999/Simplex-Method-Python-example/blob/master/img/game_theory_ex.png?raw=true)

In [ ]:
# generate the payoff matrix
# if i < j-1 or i = j+1 then i win        => -1
# else if i == j then draw                =>  0
# else if j < i-1 or j = i+1 then j win   =>  1
n = 10
A = np.ones((n, n))
for i in range(n):
  for j in range(n):
    if i < j - 1 or i == j + 1:
      A[i, j] = -1
    elif i == j:
      A[i, j] = 0

print(A)

[[ 0.  1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1.  0.  1. -1. -1. -1. -1. -1. -1. -1.]
 [ 1. -1.  0.  1. -1. -1. -1. -1. -1. -1.]
 [ 1.  1. -1.  0.  1. -1. -1. -1. -1. -1.]
 [ 1.  1.  1. -1.  0.  1. -1. -1. -1. -1.]
 [ 1.  1.  1.  1. -1.  0.  1. -1. -1. -1.]
 [ 1.  1.  1.  1.  1. -1.  0.  1. -1. -1.]
 [ 1.  1.  1.  1.  1.  1. -1.  0.  1. -1.]
 [ 1.  1.  1.  1.  1.  1.  1. -1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1. -1.  0.]]


![alt text](https://github.com/tadangkhoa1999/Simplex-Method-Python-example/blob/master/img/game_theory_1.png?raw=true)![alt text](https://github.com/tadangkhoa1999/Simplex-Method-Python-example/blob/master/img/game_theory_2.png?raw=true)

In [ ]:
import cplex

# ============================================================
# The Linear Programming problem displayed bellow is as:
#                  min z = cx
#    subject to:      Nx = b
# ============================================================

# Input all the data and parameters here
num_decision_var = n + 1
num_constraints = n + 1

In [ ]:
# Create matrix N for linear program
N = np.append(-A, np.ones((n, 1)), axis=1)
N = np.append(N, np.ones((1, n+1)), axis=0)
N[n, n] = 0
print(N)
N = N.astype(np.float).tolist()

[[-0. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1. -0. -1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [-1.  1. -0. -1.  1.  1.  1.  1.  1.  1.  1.]
 [-1. -1.  1. -0. -1.  1.  1.  1.  1.  1.  1.]
 [-1. -1. -1.  1. -0. -1.  1.  1.  1.  1.  1.]
 [-1. -1. -1. -1.  1. -0. -1.  1.  1.  1.  1.]
 [-1. -1. -1. -1. -1.  1. -0. -1.  1.  1.  1.]
 [-1. -1. -1. -1. -1. -1.  1. -0. -1.  1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1. -0. -1.  1.]
 [-1. -1. -1. -1. -1. -1. -1. -1.  1. -0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]]


In [ ]:
# Create vector b with all 0 and last element is 1
b = np.zeros((n+1,))
b[-1] = 1
print(b)
b = b.astype(np.float).tolist()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [ ]:
# Create vector c with all 0 and last elemet is -1 (for maximum)
c = np.zeros((n+1,))
c[-1] = -1
print(c)
c = c.astype(np.float).tolist()

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]


In [ ]:
# Create array of constraint type
# 'L', 'G', 'E' are Less, Greater, Equal
# All is less and the last one is equal
constraint_type = list()
for i in range(n):
  constraint_type.append("L")

constraint_type.append("E")
print(constraint_type)

['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'E']


In [ ]:
# Establish the Linear Programming Model
myProblem = cplex.Cplex()

# Add the decision variables and set their lower bound and upper bound (if necessary)
myProblem.variables.add(names= ["x"+str(i) for i in range(num_decision_var)])
for i in range(num_decision_var):
    myProblem.variables.set_lower_bounds(i, 0.0)

# Add constraints
for i in range(num_constraints):
    myProblem.linear_constraints.add(
        lin_expr= [cplex.SparsePair(ind= [j for j in range(num_decision_var)], val= N[i])],
        rhs= [b[i]],
        names = ["c"+str(i)],
        senses = [constraint_type[i]]
    )

# Add objective function and set its sense
for i in range(num_decision_var):
    myProblem.objective.set_linear([(i, c[i])])
myProblem.objective.set_sense(myProblem.objective.sense.minimize)

# Solve the model and print the answer
myProblem.solve()
print(myProblem.solution.get_values())

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.02 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual infeasibility =             1.000000
Iteration:     3   Dual objective     =             0.000000
[0.3333333333333334, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# **Part 3: Integer Programming (Branch-and-Bound)**

#### 1) Generate input matrices of Branch-and-Bound program in matrix form.

In [ ]:
def generate_matrix(m=False, n=False):              # all is interger in normal distribution
  # auto generate m, n
  if not m or not n:
    m, n = randint(2, 15), randint(2, 15)

  # random matrix size m*n with range [-25, 25]
  N = np.round(np.random.normal(0, 25, size=(m, n)))

  # create m*m matrix with ones on the diagonal and zeros elsewhere.
  B = np.eye(m)

  # create indices array
  N_ind = np.arange(1, n+1)
  B_ind = np.arange(n + 1, n+m+1)

  # create basic variables and non-basic variables
  b = np.round(np.random.normal(0, 7, size=(m, 1)))
  c_N = np.round(np.random.normal(0, 25, size=(n, 1)))

  return B, N, B_ind, N_ind, b, c_N

#### 2) Write a code to solve the Branch-and-Bound.

##### Function


In [ ]:
def is_integer_num(n):                                    # Check interger
  if isinstance(n, int):
    return True
  if isinstance(n, float):
    return n.is_integer()
  return False

def find_float(sol_x):                                    # Find float position
  for i, x in enumerate(sol_x):
    if not is_integer_num(x):
      return (i+1, x)
  return (-1, None)

def create_LP_relaxation(B, N, B_ind, N_ind, b, c_N, pos_float, x):
  m, n = N.shape
  B = np.eye(m+1)
  B_ind = np.arange(n + 1, n+m+2)
  b1  = np.append(b, math.floor(x)).reshape((-1, 1))
  b2  = np.append(b, -math.ceil(x)).reshape((-1, 1))
  e_i = np.zeros((1, n))
  e_i[0, pos_float-1] = 1
  N1 = np.append(N, e_i, axis=0)
  N2 = np.append(N, -e_i, axis=0)
  return ((B, N1, B_ind, N_ind, b1, c_N), (B, N2, B_ind, N_ind, b2, c_N))

def branch_and_bound(B, N, B_ind, N_ind, b, c_N):
  # Create tree
  tree = []
  tree.append((B, N, B_ind, N_ind, b, c_N))

  # Solve
  max = -1                                                  # max of problem (interger)
  sol_x = None
  while tree:                                               # check tree is empty
    # pop node and solve LP
    B, N, B_ind, N_ind, b, c_N = tree.pop(0)
    sol = two_phase_simplex_method(B, N, B_ind, N_ind, b, c_N)
    if sol['status'] == 'Unbounded':
      return {'status':'Unbounded', 'x':None}               # if this LP_relaxation is Unbounded => Unbounded
    elif sol['status'] == 'Infeasible':                     # elif this LP_relaxation is Infeasible => continue
      continue
    
    # if LP_relaxation has solution
    LP_relaxation_max = np.dot(c_N.flatten(), sol['x'].flatten())
    if LP_relaxation_max <= max:                           # check if float maximum is less than our maximum
      continue

    pos_float, x = find_float(sol['x'])
    if pos_float > 0:                                       # if LP_relaxation has solution and has float x
      for i in create_LP_relaxation(B, N, B_ind, N_ind, b, c_N, pos_float, x):
        tree.append(i)
    else:                                                   # if LP_relaxation has solution and all is interger
      max = LP_relaxation_max
      sol_x = sol['x']
    
  if sol_x is None:                                         # if can't find any solution
    return {'status':'Infeasible', 'x':None}

  return {'status':'Optimal', 'x':sol_x}

##### Example
![alt text](https://github.com/tadangkhoa1999/Simplex-Method-Python-example/blob/master/img/Branch_and_Bound_ex.png?raw=true)

In [ ]:
m, n = 2, 2
B = np.eye(m)
N = np.array([[10, 7], [1, 1]])
N_ind = np.arange(1, n+1)
B_ind = np.arange(n + 1, n+m+1)
x_B = np.array([[40], [5]])
c_N = np.array([[17], [12]])

sol = branch_and_bound(B, N, B_ind, N_ind, x_B, c_N)
if sol['status'] == 'Optimal':
  print(sol['x'])
else:
  print(sol['status'])

[4. 0.]


#### 3) Solve the generated LP by a tool such as CVX, Cplex, ... to check your code.

##### Function

In [ ]:
# install library
!pip install pulp

     |████████████████████████████████| 40.6MB 84kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for amply: filename=amply-0.1.2-cp36-none-any.whl size=16572 sha256=2875115072bba06b4358b33517939aa682e370326823654fc1f0df75da8d8a1e
  Stored in directory: /root/.cache/pip/wheels/84/18/f7/e5c3ed13ed5bb721763f77d4a924331d59ef115ce61c9d26eb
Successfully built amply


In [ ]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpInteger

def pulp_branch_and_bound(B, N, B_ind, N_ind, x_B, c_N):
  # Create the model
  model = LpProblem(name="small-problem", sense=LpMaximize)

  # Initialize the decision variables
  X = []                                                  # list of variables
  for i in range(c_N.shape[0]):
    X.append(LpVariable(name="x"+str(i+1), lowBound=0, cat=LpInteger))

  # Add the constraints to the model
  for i, a in enumerate(N):                               # contrain_i
    constraint = 0
    for j, a_ij in enumerate(a):
      constraint += a_ij * X[j]
    model += (constraint <= x_B[i])

  # Add the objective function to the model
  model += lpSum([c * x for c, x in zip(c_N.flatten(), X)])

  # Solve the problem
  status = model.solve()
  print(f"status: {model.status}, {LpStatus[model.status]}")
  for var in model.variables():
    print(f"{var.name}: {var.value()}")

##### Example

In [ ]:
m, n = 2, 2
B = np.eye(m)
N = np.array([[10, 7], [1, 1]])
N_ind = np.arange(1, n+1)
B_ind = np.arange(n + 1, n+m+1)
x_B = np.array([[40], [5]])
c_N = np.array([[17], [12]])

pulp_branch_and_bound(B, N, B_ind, N_ind, x_B, c_N)

status: 1, Optimal
x1: 4.0
x2: 0.0


#### 4) Repeat (1)-(2) one hundred times to check the mean and standard deviation of rune time of your code.

In [ ]:
%%capture
mycode_time = []
tool_time = []
for i in range(50):
  B, N, B_ind, N_ind, b, c_N = generate_matrix()
  begin_time = datetime.datetime.now().timestamp()
  try:
    sol = branch_and_bound(B, N, B_ind, N_ind, b, c_N)
  except:
    print('error!')
    print(B, N, B_ind, N_ind, b, c_N)
  mycode_time.append(datetime.datetime.now().timestamp() - begin_time)

  begin_time = datetime.datetime.now().timestamp()
  try:
    pulp_branch_and_bound(B, N, B_ind, N_ind, x_B, c_N)
  except:
    print('error!')
    print(B, N, B_ind, N_ind, b, c_N)
  tool_time.append(datetime.datetime.now().timestamp() - begin_time)

In [ ]:
print("My code mean time:", np.mean(mycode_time))
print("My code standard deviation:", np.std(mycode_time))
print("Tool mean time:", np.mean(tool_time))
print("Tool standard deviation:", np.std(tool_time))

My code mean time: 0.0011751127243041992
My code standard deviation: 0.001474042409928471
Tool mean time: 0.001769728660583496
Tool standard deviation: 0.0030153585581455884
